In [1]:
from music21 import *
import os 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import torch
from torch import tensor
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf
import torch.nn.functional as F
import math
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [2]:
import pandas as pd
from ast import literal_eval

notes_df = pd.read_csv ('Dataset/notes.csv')
train_df = pd.read_csv ('Dataset/trainset.csv')
validation_df = pd.read_csv ('Dataset/validationset.csv')

data_train = train_df[['x_tr','future']].to_numpy()
data_validation = validation_df[['x_val','future']].to_numpy()

x_tr_string = data_train[:,0]
y_tr_string = data_train[:,1]
x_tr = []
y_tr = []
for i in x_tr_string:

    b = "[]\n"
    for char in b:
        i = i.replace(char, "")
    input_x_tr = [int(j) for j in i.split()]
    x_tr.append(input_x_tr)

for i in y_tr_string:

    b = "[]\n"
    for char in b:
        i = i.replace(char, "")
    input_y_tr = [int(j) for j in i.split()]
    y_tr.append(input_y_tr)
x_tr = np.array(x_tr)
y_tr = np.array(y_tr)
    
# print(x_tr.shape,y_tr.shape)

x_val_string = data_validation[:,0]
y_val_string = data_validation[:,1]
x_val = []
y_val = []
for i in x_val_string:

    b = "[]\n"
    for char in b:
        i = i.replace(char, "")
    input_x_val = [int(j) for j in i.split()]
    x_val.append(input_x_val)

for i in y_val_string:

    b = "[]\n"
    for char in b:
        i = i.replace(char, "")
    input_y_val = [int(j) for j in i.split()]
    y_val.append(input_y_val)
x_val = np.array(x_val)
y_val = np.array(y_val)

# print(x_val.shape,y_val.shape)

notes_ = notes_df.to_numpy()[:,1]
unique_notes_reverse = dict(enumerate(notes_.flatten(), 1))
unique_notes = {value : key for (key, value) in unique_notes_reverse.items()}

In [3]:
from MusicDataset import *
batch_size = 8
train_set = MusicDataset(x_tr,y_tr)
trainloader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

In [4]:
val_set = MusicDataset(x_val,y_val)
validationloader = torch.utils.data.DataLoader(val_set, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

In [7]:
# Define the parameters for your new task
from Models import  gru
input_size = len(unique_notes)  # Assuming this is defined somewhere in your code
hidden_size = 32
num_layers = 2
output_size = 8  # Adjust this according to your task
embedding_dimension = 100  # Choose an appropriate value

# Initialize GRUModel with appropriate arguments
Net = gru.GRUSequence(num_classes=input_size, seq_length=None, embedding_dimension=embedding_dimension,
                      num_layers=num_layers, hidden_size_gru=hidden_size)

# Move the model to the device if you're using GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Net.to(device)



GRUSequence(
  (embedding): Embedding(182, 100)
  (gru): GRU(100, 32, num_layers=2, batch_first=True)
  (linear1): Linear(in_features=32, out_features=128, bias=True)
  (relu): ReLU()
  (linear2): Linear(in_features=128, out_features=128, bias=True)
  (linear3): Linear(in_features=128, out_features=182, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [8]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(Net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

In [9]:
import os
import time

baseDir = r'C:\Users\Dell\music_generation/Music-Gen-AI'
expDir = os.path.join(baseDir, 'trained_model_cache', time.strftime("%d_%m_%Y_%H_%M"))
lossDir = os.path.join(expDir, 'loss')
os.makedirs(expDir, exist_ok=True)
os.makedirs(lossDir, exist_ok=True)
print(baseDir, expDir)

C:\Users\Dell\music_generation/Music-Gen-AI C:\Users\Dell\music_generation/Music-Gen-AI\trained_model_cache\27_04_2024_23_24


In [10]:
loss_history = []
best_val_acc = 0
training_mode_list = ["Next_Predict_Mode","Future_Predict_Mode"]
training_mode = "Next_Predict_Mode"
# training_mode = "Future_Predict_Mode"
future_preds = 5

In [11]:
import timeit

''' TRAINING THE MODEL '''
correct_preds = 0
total_preds = 0
for epoch in range(20):
    running_loss = 0.0
    start = timeit.default_timer()
    for i, data in enumerate(trainloader, 0):
        input, label = data
        optimizer.zero_grad()
        label = label[:, 0].long()  # Ensure label type is Long
        if training_mode == training_mode_list[0]:
            # No need to add a batch dimension to the input tensor
            output = Net(input.to(device))
            loss = criterion(output.to(device), label.to(device))
            loss.backward()
            optimizer.step()
            total_preds += input.shape[0]
            correct_preds += torch.sum(torch.argmax(output, 1) == label.to(device))

            running_loss += loss.item()
        elif training_mode == training_mode_list[1]:
            cumm_output = torch.zeros(0, len(unique_notes)).to(device)
            cumm_label = np.array([], dtype=int)
            for k in range(future_preds):
                # No need to add a batch dimension to the input tensor
                output = Net(input.to(device))
                cumm_output = torch.cat((cumm_output, output))
                cumm_label = np.concatenate((cumm_label, label[:, k]))
                next_preds = np.argmax(output.cpu().detach().numpy(), axis=1)
                input = input.cpu().detach().numpy()
                input = torch.from_numpy(np.array([np.append(j, next_preds[ind])
                                                    for ind, j in enumerate(input)])[:, 1:])
            loss = criterion(cumm_output, torch.from_numpy(cumm_label).to(device))
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        if (i % 300 == 299):
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 300))
            loss_history.append(running_loss / 300)
            running_loss = 0.0
    stop = timeit.default_timer()
    print(f"Training Time for Epoch :{epoch} \n {stop-start} seconds\n")
    train_acc = float(correct_preds) / float(total_preds) * 100
    trainreport = "Training Accuracy : \n correct predictions  : {} \n total predictions : {} \n Training Accuracy : {} \n ------------------------\n".format(
        correct_preds, total_preds, train_acc)
    print(trainreport)
    correct_preds = 0
    total_preds = 0
    val_loss = 0
    start = timeit.default_timer()
    for i, data in enumerate(validationloader, 0):
        input, label = data
        label = label[:, 0].long()  # Ensure label type is Long
        if training_mode == training_mode_list[0]:
            # No need to add a batch dimension to the input tensor
            output = Net(input.to(device))
            loss = criterion(output.to(device), label.to(device))
            total_preds += input.shape[0]
            correct_preds += torch.sum(torch.argmax(output, 1) == label.to(device))
            val_loss += loss.item()

        elif training_mode == training_mode_list[1]:
            cumm_output = torch.zeros(0, len(unique_notes)).to(device)
            cumm_label = np.array([], dtype=int)
            for k in range(future_preds):
                # No need to add a batch dimension to the input tensor
                output = Net(input.to(device))
                cumm_output = torch.cat((cumm_output, output))
                cumm_label = np.concatenate((cumm_label, label[:, k]))
                next_preds = np.argmax(output.cpu().detach().numpy(), axis=1)
                input = input.cpu().detach().numpy()
                input = torch.from_numpy(np.array([np.append(j, next_preds[ind])
                                                    for ind, j in enumerate(input)])[:, 1:])
            loss = criterion(cumm_output, torch.from_numpy(cumm_label).to(device))

            val_loss += loss.item()
    stop = timeit.default_timer()
    print(f"Validation Time for Epoch :{epoch} \n {stop-start} seconds\n")
    val_acc = float(correct_preds) / float(total_preds) * 100
    val_loss = float(val_loss) / float(total_preds)
    valreport = "Validation Accuracy : \n correct predictions  : {} \n total predictions : {} \n Val-Loss : {} \n Val-Accuracy :{}----------------------------------\n".format(
        correct_preds, total_preds, val_loss, val_acc)
    print(valreport)

    if (val_acc > best_val_acc):
        print('Updating the model')
        best_val_acc = val_acc
        checkpoint = {
            "epoch": epoch,
            "model_state_dict": Net.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "train_acc": train_acc,
            "valid_acc": val_acc
        }
        torch.save(checkpoint, os.path.join(expDir, 'checkpoint1.tar'))
        torch.save(Net, os.path.join(expDir, 'model1.pth'))

    correct_preds = 0
    total_preds = 0
.00000000000000000
print('Finished Training')


[1,   300] loss: 4.897
[1,   600] loss: 4.659
[1,   900] loss: 4.580
[1,  1200] loss: 4.516
[1,  1500] loss: 4.456
[1,  1800] loss: 4.431
[1,  2100] loss: 4.398
[1,  2400] loss: 4.348
[1,  2700] loss: 4.305
[1,  3000] loss: 4.280
[1,  3300] loss: 4.192
[1,  3600] loss: 4.183
[1,  3900] loss: 4.167
[1,  4200] loss: 4.161
[1,  4500] loss: 4.157
[1,  4800] loss: 4.110
[1,  5100] loss: 4.041
[1,  5400] loss: 4.060
[1,  5700] loss: 4.124
[1,  6000] loss: 4.038
[1,  6300] loss: 4.074
[1,  6600] loss: 4.010
Training Time for Epoch :0 
 161.5344055999958 seconds

Training Accuracy : 
 correct predictions  : 3828 
 total predictions : 54873 
 Training Accuracy : 6.976108468645783 
 ------------------------

Validation Time for Epoch :0 
 23.177757800003747 seconds

Validation Accuracy : 
 correct predictions  : 1504 
 total predictions : 15678 
 Val-Loss : 0.5024923370845282 
 Val-Accuracy :9.593060339328996----------------------------------

Updating the model
[2,   300] loss: 4.022
[2,   600]

[9,  6300] loss: 3.599
[9,  6600] loss: 3.541
Training Time for Epoch :8 
 198.33666549999907 seconds

Training Accuracy : 
 correct predictions  : 7841 
 total predictions : 54873 
 Training Accuracy : 14.289359065478468 
 ------------------------

Validation Time for Epoch :8 
 47.48443590000534 seconds

Validation Accuracy : 
 correct predictions  : 2116 
 total predictions : 15678 
 Val-Loss : 0.464925678610604 
 Val-Accuracy :13.496619466768719----------------------------------

Updating the model
[10,   300] loss: 3.595
[10,   600] loss: 3.553
[10,   900] loss: 3.573
[10,  1200] loss: 3.593
[10,  1500] loss: 3.549
[10,  1800] loss: 3.595
[10,  2100] loss: 3.616
[10,  2400] loss: 3.627
[10,  2700] loss: 3.588
[10,  3000] loss: 3.636
[10,  3300] loss: 3.533
[10,  3600] loss: 3.570
[10,  3900] loss: 3.556
[10,  4200] loss: 3.608
[10,  4500] loss: 3.603
[10,  4800] loss: 3.624
[10,  5100] loss: 3.462
[10,  5400] loss: 3.575
[10,  5700] loss: 3.611
[10,  6000] loss: 3.571
[10,  6300] 

[18,   300] loss: 3.526
[18,   600] loss: 3.514
[18,   900] loss: 3.519
[18,  1200] loss: 3.545
[18,  1500] loss: 3.522
[18,  1800] loss: 3.570
[18,  2100] loss: 3.580
[18,  2400] loss: 3.608
[18,  2700] loss: 3.557
[18,  3000] loss: 3.585
[18,  3300] loss: 3.494
[18,  3600] loss: 3.560
[18,  3900] loss: 3.529
[18,  4200] loss: 3.578
[18,  4500] loss: 3.558
[18,  4800] loss: 3.591
[18,  5100] loss: 3.455
[18,  5400] loss: 3.549
[18,  5700] loss: 3.549
[18,  6000] loss: 3.481
[18,  6300] loss: 3.575
[18,  6600] loss: 3.525
Training Time for Epoch :17 
 156.17574310000055 seconds

Training Accuracy : 
 correct predictions  : 8213 
 total predictions : 54873 
 Training Accuracy : 14.967288101616461 
 ------------------------

Validation Time for Epoch :17 
 23.143317699999898 seconds

Validation Accuracy : 
 correct predictions  : 2112 
 total predictions : 15678 
 Val-Loss : 0.4640403050429846 
 Val-Accuracy :13.471106008419442----------------------------------

[19,   300] loss: 3.524
[

In [1]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(loss_history)
plt.savefig(lossDir + '/loss_history.png')

NameError: name 'loss_history' is not defined

<Figure size 640x480 with 0 Axes>